In [68]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import openpyxl

In [69]:
df_Jan = pd.read_csv('../New system/Data_elements_(Cases)_jan.csv')
df_Feb = pd.read_csv('../New system/Data_elements_(Cases)_feb.csv')
df_Mar = pd.read_csv('../New system/Data_elements_(Cases)_march.csv')
df_Apr = pd.read_csv('../New system/Data_elements_(Cases)_april.csv')

In [70]:
def data_Quality(indicator):
    df = pd.DataFrame(districts).reset_index(drop=True)

    Jan = df_Jan[df_Jan['Data']==indicator].drop('Data', axis=1)
    Feb = df_Feb[df_Feb['Data']==indicator].drop('Data', axis=1)
    March = df_Mar[df_Mar['Data']==indicator].drop('Data', axis=1)
    April = df_Apr[df_Apr['Data']==indicator].drop('Data', axis=1)
    
    #print(Jan.shape[0]+ Feb.shape[0]+ March.shape[0] + April.shape[0])

    df_final = Jan.merge(Feb,on='Organisation unit', how='outer').merge(March,on='Organisation unit', how='outer').merge(April,on='Organisation unit', how='outer')
    #df_final.columns = ["Districts", "Jan_"+indicator, "Feb_"+indicator, "Mar_"+indicator, "Apr_"+indicator]
    columns = [('index',"Districts"), (indicator,"Jan"), (indicator,"Feb"),(indicator,"Mar"),(indicator,"Avr")]
    df_final.columns=pd.MultiIndex.from_tuples(columns)
    return df_final


In [71]:
data_Quality('033B-CD01a. Malaria (diagnosed)  - Cases')

index 033B-CD01a. Malaria (diagnosed)  - Cases           \
               Districts                                      Jan      Feb   
0        Amuria District                                   2025.0   6710.0   
1         Amuru District                                   5622.0   9317.0   
2       Sironko District                                    398.0   1964.0   
3        Kitgum District                                   1944.0   6135.0   
4    Kitagwenda District                                    537.0   2732.0   
..                   ...                                      ...      ...   
130     Karenga District                                     46.0   1614.0   
131      Mayuge District                                   2625.0  12442.0   
132      Moroto District                                   1285.0   3315.0   
133   Sembabule District                                   3671.0   4672.0   
134      Amudat District                                      NaN   1328.0   

                      
        Mar      Avr  
0    7877.0   9388.0  
1    8246.0  10237.0  
2    1868.0   1841.0  
3    6420.0   7846.0  
4    1926.0   2765.0  
..      ...      ...  
130  1607.0   1470.0  
131  9096.0   7701.0  
132  1734.0   2439.0  
133  2309.0   2545.0  
134   568.0   1388.0  

[135 rows x 5 columns]

# Data elements

In [60]:
districts = df_Feb[df_Feb['Data']=='033B-CD01a. Malaria (diagnosed)  - Cases']['Organisation unit']

In [62]:
multi_index=pd.MultiIndex.from_tuples(('index','Districts'))
data_elements = pd.DataFrame(data=districts,columns=multi_index).reset_index(drop=True)

In [72]:
#data_elements = pd.DataFrame(districts,columns=multi_index).reset_index(drop=True)
#data_elements.columns = ['Districts']
#data_elements.columns = pd.MultiIndex.from_tuples(('index','Districts'))
indicators = df_Jan['Data'].unique()
for indicator in indicators:
    df_months = data_Quality(indicator)
    data_elements = pd.merge(data_elements, df_months, how='left', on=['index','Districts'])  

#data_elements.set_index('Districts', inplace=True,drop=True)

C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (5 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


ValueError: The column label 'index' is not unique.
For a multi-index, the label must be a tuple with elements corresponding to each level.

In [6]:
# Proprtion of of districts with reported data for each data element
null_sum1= data_elements.count()/135

# Indicators 

In [7]:
df_Jan = pd.read_csv('../New system/Indicator_January.csv')
df_Feb = pd.read_csv('../New system/Indicator_Feb.csv')
df_Mar = pd.read_csv('../New system/Indicator_march.csv')
df_Apr = pd.read_csv('../New system/Indicator_April.csv')

In [8]:
districts = df_Feb[df_Feb['Data']=='EPI - BCG doses given']['Organisation unit'].unique()

In [9]:
data_Indicators = pd.DataFrame()
data_Indicators = pd.DataFrame(districts).reset_index(drop=True)
data_Indicators.columns = ['Districts']
indicators = df_Jan['Data'].unique()
for indicator in indicators:
    df_months = data_Quality(indicator)
    data_Indicators = pd.merge(data_Indicators, df_months, how='left', on='Districts')

In [10]:
data_Indicators.drop_duplicates('Districts',inplace=True,keep='first',ignore_index=True) # This shouldnt be needed, there might be a bug
data_Indicators.set_index('Districts', inplace=True,drop=True)

In [11]:
null_sum2= data_Indicators.count()/135

# Mixing both

In [12]:
null_sum=pd.concat([null_sum1,null_sum2])

In [13]:
null_sum

Jan_033B-TB04. New and Relapse TB cases started on TX                           0.311111
Feb_033B-TB04. New and Relapse TB cases started on TX                           0.866667
Mar_033B-TB04. New and Relapse TB cases started on TX                           0.903704
Apr_033B-TB04. New and Relapse TB cases started on TX                           0.896296
Jan_033B-RV07. RE Blister of 28 tablets                                         0.651852
                                                                                  ...   
Apr_TB-Weekly - Percentage  of samples tested with GeneXpert out of expected    0.814815
Jan_TB-Weekly - Percentage  of weekly attendance screened for TB                0.681481
Feb_TB-Weekly - Percentage  of weekly attendance screened for TB                1.000000
Mar_TB-Weekly - Percentage  of weekly attendance screened for TB                1.000000
Apr_TB-Weekly - Percentage  of weekly attendance screened for TB                1.000000
Length: 1444, dtype: 

In [14]:
null_sum_df = pd.DataFrame(null_sum)
null_sum_df.columns = ['% of data available']

In [15]:
null_sum_df.reset_index(inplace=True)
null_sum_df['month']=null_sum_df['index'].apply(lambda x: x[0:3])
null_sum_df['indicator']=null_sum_df['index'].apply(lambda x: x[4:])

In [16]:
null_av_df=null_sum_df.groupby('indicator')['% of data available'].mean()

# Adding target data 

In [17]:
null_av_df

indicator
033B-AP01. OPD New                                                          0.998148
033B-AP02. Total OPD                                                        0.998148
033B-AP03. Expected eMTCT Mothers in Appt                                   0.979630
033B-AP04. eMTCT Missed Appointments                                        0.937037
033B-CD01a. Malaria (diagnosed)  - Cases                                    0.998148
                                                                              ...   
TB-Weekly - Percentage  of new and relapse cases started on treatment       0.742593
TB-Weekly - Percentage  of presumptives identified                          0.883333
TB-Weekly - Percentage  of samples rejected                                 0.261111
TB-Weekly - Percentage  of samples tested with GeneXpert out of expected    0.664815
TB-Weekly - Percentage  of weekly attendance screened for TB                0.920370
Name: % of data available, Length: 361, dtype: float64

In [18]:
target = pd.read_excel('../New system/Data summary.xlsx',sheet_name='target')

In [19]:
target = pd.merge(target,null_av_df,left_on='Var',right_on='indicator',how='left')
target['keep']=(target['% of data available']>0.7)

In [21]:
with ExcelWriter('../New system/Data summary.xlsx',mode='a') as writer:
   target.to_excel(writer,sheet_name='target_availability')

In [36]:
list(target.loc[target['keep']==1,'Var'])

['EPI - BCG doses given',
 'EPI - BCG to MR 1 dropout rate (%)',
 'EPI - DPT 1 to DPT3 dropout rate (%)',
 '033B-CD01a. Malaria (diagnosed)  - Cases',
 'NUT: Percentage of children/babies born with low birth weight (<2.5kg)',
 'NUT: Percentage of pregnant women with Hb level <10g/dl or diagnosed with Anaemia at ANC 1st visit',
 'ANC - Proportion of pregnant women attending ANC 1 in first trimester',
 'MNCH - Facility Stillbirth rate (Fresh Stillbirth)',
 'MNCH - Facility Stillbirth rate (Macerated Stillbirth)',
 'MNCH - Pre-discharge neonatal mortality rate',
 '033B-TP01. No. of Adults ART clients initiated TPT']